# Interactive feature matching

In [1]:
import sys
sys.path.append('..')

from multiview_stitcher import spatial_image_utils as si_utils

from muvis_align.MVSRegistration import MVSRegistration
from muvis_align.util import dir_regex
from muvis_align.image.util import get_sim_physical_size, draw_keypoints_matches

## Setup parameters

In [2]:
params = {
	'general':
	{
        'ui': 'mpl',
        'debug': True
	},
	'operations':
	[
		{
			'operation': 'register',
			'input': '../data/S000/*.tiff',
			'source_metadata':
			{
				'scale': {'x': 0.032, 'y': 0.032},
				'position': {'y': 'fn[-3]*24', 'x': 'fn[-2]*24'}
			},
            'use_orthogonal_pairs': True,
            'n_parallel_pairwise_regs': 1,
			'output': '../../output/'
		}
	]
}


## Initialise muvis-align, initialise sims, and pre-process

In [3]:
operation_params = params['operations'][0]
filenames = dir_regex(operation_params['input'])

reg = MVSRegistration(params['general'])
reg.init_operation('', filenames, operation_params)
sims = reg.init_sims()
norm_sims, _ = reg.preprocess(sims, operation_params)

for label, sim in zip(reg.file_labels, sims):
    print(label, si_utils.get_origin_from_sim(sim), get_sim_physical_size(sim))

000_000 {'y': 0.0, 'x': 0.0} {'y': 25.6, 'x': 25.6}
000_001 {'y': 0.0, 'x': 24.0} {'y': 25.6, 'x': 25.600000000000023}
001_000 {'y': 24.0, 'x': 0.0} {'y': 25.600000000000023, 'x': 25.6}
001_001 {'y': 24.0, 'x': 24.0} {'y': 25.600000000000023, 'x': 25.600000000000023}


## Get overlap between tiles

In [4]:
%matplotlib inline
overlap1, overlap2 = reg.get_overlap_images(norm_sims[0], norm_sims[1], reg.source_transform_key)
overlap1, overlap2 = overlap1.squeeze().compute(), overlap2.squeeze().compute()
print('Overlap size [um]:', overlap1.shape * si_utils.get_spacing_from_sim(sims[0], asarray=True))

draw_keypoints_matches(overlap1, [], overlap2, [])

(<Figure size 4800x2400 with 1 Axes>, <Axes: >)

## Interactive: Set up feature extraction and matching, and show results

### You can modify the parameters and rerun this cell to update results, e.g.:

**transform_type**: affine, euclidean, translation

**name**: sift, orb

In [5]:
%matplotlib inline
reg.params.update({
    'transform_type': 'affine',
    'method': {
        'name': 'sift',
        'gaussian_sigma': 1,
        'max_keypoints': 5000,
        'inlier_threshold_factor': 0.05,
        'max_trials': 1000,
        'ransac_iterations': 3
    }
})

reg_method, pairwise_reg_func, pairwise_reg_func_kwargs = reg.create_registration_method(sims[0])
result = pairwise_reg_func(overlap1, overlap2)
print('transform:\n', result['affine_matrix'])

transform:
 [[ 9.99997537e-01 -2.21928887e-03 -3.34173975e+01]
 [ 2.21928887e-03  9.99997537e-01 -5.26946222e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Run registration on all tiles, showing results corresponding to each tile overlap

In [ ]:
%matplotlib inline
reg.register(sims, norm_sims)